In [59]:
import torch
ckpt_path = "/mnt/wjr/HTS-Audio-Transformer/ckpt/HTSAT_AudioSet_Saved_1.ckpt"
state_dict = torch.load(ckpt_path)['state_dict']
new_state_dict = {}
for key in state_dict.keys():
    new_key = key[10:]
    new_state_dict[new_key] = state_dict[key]

In [4]:
print(new_state_dict.keys())

dict_keys(['spectrogram_extractor.stft.conv_real.weight', 'spectrogram_extractor.stft.conv_imag.weight', 'logmel_extractor.melW', 'bn0.weight', 'bn0.bias', 'bn0.running_mean', 'bn0.running_var', 'bn0.num_batches_tracked', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'patch_embed.norm.weight', 'patch_embed.norm.bias', 'layers.0.blocks.0.norm1.weight', 'layers.0.blocks.0.norm1.bias', 'layers.0.blocks.0.attn.relative_position_bias_table', 'layers.0.blocks.0.attn.relative_position_index', 'layers.0.blocks.0.attn.qkv.weight', 'layers.0.blocks.0.attn.qkv.bias', 'layers.0.blocks.0.attn.proj.weight', 'layers.0.blocks.0.attn.proj.bias', 'layers.0.blocks.0.norm2.weight', 'layers.0.blocks.0.norm2.bias', 'layers.0.blocks.0.mlp.fc1.weight', 'layers.0.blocks.0.mlp.fc1.bias', 'layers.0.blocks.0.mlp.fc2.weight', 'layers.0.blocks.0.mlp.fc2.bias', 'layers.0.blocks.1.attn_mask', 'layers.0.blocks.1.norm1.weight', 'layers.0.blocks.1.norm1.bias', 'layers.0.blocks.1.attn.relative_position_bias_table',

In [60]:
new_path = "/mnt/wjr/LAVIS/lavis/models/htsat_models/ckpt/HTSAT_AudioSet_Saved_1.ckpt"
torch.save(new_state_dict, new_path)

In [ ]:
from lavis.models import load_model
model = load_model("blip2", "pretrain")

In [5]:
from PIL import Image
from lavis.processors.blip_processors import BlipImageTrainProcessor
image_path = "/mnt1/wjr/coco/test2014/COCO_test2014_000000000001.jpg"

image = Image.open(image_path).convert("RGB")

vis_processor = BlipImageTrainProcessor(image_size=224)

image_input = vis_processor(image).unsqueeze(0)

In [6]:
print(image_input.shape)

torch.Size([1, 3, 224, 224])


In [15]:
import torch
image_embeds = model.ln_vision(model.visual_encoder(image_input))
image_atts = torch.ones(image_embeds.size()[:-1], dtype=torch.long)

query_tokens = model.query_tokens.expand(image_embeds.shape[0], -1, -1)

query_output = model.Qformer.bert(
        query_embeds=query_tokens,
        encoder_hidden_states=image_embeds,
        encoder_attention_mask=image_atts,
        use_cache=True,
        return_dict=True,
    )
print(image_embeds.shape)

torch.Size([1, 257, 1408])


In [1]:
import torch
from lavis.models.blap_models.blap_qformer import BlapQformer
model = BlapQformer().to("cuda")

/home/adgroup/anaconda3/envs/dcase2022/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/adgroup/anaconda3/envs/dcase2022/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484683044/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
from lavis.processors.blap_processors import BlapAudioProcessor
audio_processor = BlapAudioProcessor(sample_rate=32000, max_sec=30)

In [43]:
audio_feats = torch.load("/mnt/wjr/LAVIS/test_dir/audio_features.pth")
torch.set_printoptions(threshold=torch.inf)
print(torch.any(torch.isnan(audio_feats)))
#print(audio_feats)

tensor(True, device='cuda:0')


In [23]:
data = torch.load("/mnt/wjr/LAVIS/test_dir/logmel.pth")
torch.set_printoptions(threshold=torch.inf)
# print(torch.any(torch.isnan(data)))
for i in range(100):
    if torch.any(torch.isnan(data[i,:,:])).item():
        print(i)
# print(data[10,:,:])

10
12
13
21
26
28
35
47
50
64


In [1]:
import torch
from lavis.models.htsat_models.htsat import HTSAT_Swin_Transformer
import lavis.models.htsat_models.config as htsat_config
file_path = '/mnt/wjr/LAVIS/lavis/models/htsat_models/ckpt/HTSAT_AudioSet_Saved_1.ckpt'
state_dict = torch.load(file_path)
audio_encoder = HTSAT_Swin_Transformer(
            spec_size=htsat_config.htsat_spec_size,
            patch_size=htsat_config.htsat_patch_size,
            in_chans=1,
            num_classes=htsat_config.classes_num,
            window_size=htsat_config.htsat_window_size,
            config=htsat_config,
            depths=htsat_config.htsat_depth,
            embed_dim=htsat_config.htsat_dim,
            patch_stride=htsat_config.htsat_stride,
            num_heads=htsat_config.htsat_num_head)
audio_encoder.load_state_dict(state_dict)
audio_encoder = audio_encoder.eval()
audio_encoder.to("cuda")

/home/adgroup/anaconda3/envs/dcase2022/lib/python3.8/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/adgroup/anaconda3/envs/dcase2022/lib/python3.8/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484683044/work/aten/src/ATen/native/TensorShape.cpp:2894.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


HTSAT_Swin_Transformer(
  (spectrogram_extractor): Spectrogram(
    (stft): STFT(
      (conv_real): Conv1d(1, 513, kernel_size=(1024,), stride=(320,), bias=False)
      (conv_imag): Conv1d(1, 513, kernel_size=(1024,), stride=(320,), bias=False)
    )
  )
  (logmel_extractor): LogmelFilterBank()
  (spec_augmenter): SpecAugmentation(
    (time_dropper): DropStripes()
    (freq_dropper): DropStripes()
  )
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0): BasicLayer(
      dim=96, input_resolution=(64, 64), depth=2
      (blocks): ModuleList(
        (0): SwinTransformerBlock(
          dim=96, input_resolution=(64, 64), num_heads=4, window_size=8, shift_size=0, mlp_ratio=4.0
          (norm1): LayerNorm((96,), eps=1e-

In [24]:
import os
import random
import numpy as np
from lavis.processors.blap_processors import BlapAudioProcessor
def gen_waveform(dir, num_sample, audio_processor):
    files = os.listdir(dir)
    assert len(files) >= num_sample
    samples = files[0:100]

    waveform = None
    for i, sample in enumerate(samples):
        path = dir + "/" + sample
        y = audio_processor(path)

        if waveform is None:
            waveform = y
        else:
            waveform = np.vstack([waveform, y])

    return waveform


In [25]:
from lavis.processors.blap_processors import BlapAudioProcessor
audio_processor = BlapAudioProcessor(sample_rate=32000, max_sec=30)
waveform = gen_waveform("/mnt1/datasets/Clotho/clotho_audio_development/development", 100, audio_processor)
waveform = torch.from_numpy(waveform).to("cuda")
print(waveform.shape)

torch.Size([100, 960000])


In [26]:
x = audio_encoder.spectrogram_extractor(waveform)
x = audio_encoder.logmel_extractor(x)
print(torch.any(torch.isnan(x)))

tensor(False, device='cuda:0')


In [55]:
data = torch.load("/mnt/wjr/LAVIS/test_dir/spec.pth")
torch.set_printoptions(threshold=torch.inf)
data.to("cuda")
#spec = audio_encoder.spectrogram_extractor(data)
#print(spec.shape)
logmel = audio_encoder.logmel_extractor(data)
print(torch.any(torch.isnan(logmel)))

tensor(False, device='cuda:0')


In [56]:
new_logmel = torch.load("/mnt/wjr/LAVIS/test_dir/new_logmel.pth")
torch.set_printoptions(threshold=torch.inf)
print(torch.any(torch.isnan(new_logmel)))

tensor(False, device='cuda:0')


In [61]:
import torch
ckpt_path = "/mnt/wjr/LAVIS/lavis/models/htsat_models/ckpt/HTSAT_AudioSet_Saved_1.ckpt"
state_dict = torch.load(ckpt_path)
new_state_dict = {}
keys = ['spectrogram_extractor.stft.conv_real.weight', 'spectrogram_extractor.stft.conv_imag.weight', 'logmel_extractor.melW']
for k in keys:
    new_state_dict[k] = state_dict[k]

In [62]:
new_path = "/mnt/wjr/LAVIS/lavis/models/htsat_models/ckpt/gen_logmel.ckpt"
torch.save(new_state_dict, new_path)

In [5]:
import torch
ckpt_path = "/mnt/wjr/LAVIS/lavis/output/BLAP/Pretrain_stage1/20230720130/checkpoint_0.pth"
state_dict = torch.load(ckpt_path)["model"]
#new_state_dict = {}
#for key in state_dict.keys():
#    new_key = key[10:]
#    new_state_dict[new_key] = state_dict[key]
print(state_dict.keys())

odict_keys(['query_tokens', 'temp', 'audio_encoder.bn0.running_mean', 'audio_encoder.bn0.running_var', 'audio_encoder.bn0.num_batches_tracked', 'audio_encoder.layers.0.blocks.0.attn.relative_position_index', 'audio_encoder.layers.0.blocks.1.attn_mask', 'audio_encoder.layers.0.blocks.1.attn.relative_position_index', 'audio_encoder.layers.1.blocks.0.attn.relative_position_index', 'audio_encoder.layers.1.blocks.1.attn_mask', 'audio_encoder.layers.1.blocks.1.attn.relative_position_index', 'audio_encoder.layers.2.blocks.0.attn.relative_position_index', 'audio_encoder.layers.2.blocks.1.attn_mask', 'audio_encoder.layers.2.blocks.1.attn.relative_position_index', 'audio_encoder.layers.2.blocks.2.attn.relative_position_index', 'audio_encoder.layers.2.blocks.3.attn_mask', 'audio_encoder.layers.2.blocks.3.attn.relative_position_index', 'audio_encoder.layers.2.blocks.4.attn.relative_position_index', 'audio_encoder.layers.2.blocks.5.attn_mask', 'audio_encoder.layers.2.blocks.5.attn.relative_position